# Shapley Values in Linear SEMs
In the paper Budhathoki et. al.: Causal structure-based root cause analysis of outliers, Shapley values are used for root cause analysis. Our first question is to write out the formula for the shapley values in the case of a linear SEM. For $f$ the function of the SEM, $\tau$ used in the outleier score (e.g. identity or $\| \cdot \|$ or $\tau(x) = |x-\mu|$, which gives the z-score), $g=\tau \circ f$, the contribution of $j\mid \mathcal{I}$ is defined as
$$C(j\mid \mathcal{I}) = -\log P^{\text{rd}(N_{\mathcal{I} \cup \{j\}})} \{g(N) \geq g(n)\}+\log P^{\text{rd}(N_{\mathcal{I}})} \{g(N) \geq g(n)\}.$$
The Shapley value is
$$\phi(j)=\frac{1}{n!}\sum_{\sigma}C(j\mid \mathcal{I}_{j}^{\text{prec};\sigma})=\sum_{\mathcal{I}\subseteq U\setminus\{j\}}\frac{1}{n \binom{n-1}{|\mathcal{I}|}}C(j\mid \mathcal{I}).$$
With this setup, there is no simple formula for the Shapley value in linear SEMs, in contrast to some other papers that do not use the log transformation in their definition of contribution. 

### Comparison: Shapley values without log transformation
For example, in "Feature relevance quantification in explainable AI: A causal problem" they want to attribute feature importance to each $x_j$ (of one sample x) in a model 
$$Y \approx f(X_1, \dots, X_n).$$
Here, 
$$C(j\mid T) = \mathbb{E}(f(x_{T \cup \{j\}}, X_{(T \cup \{j\})^C})) - \mathbb{E}(f(x_T, X_{T^C})).$$
Hence, if $f(x) = a_1 x_1 + \dots a_n x_n$ is linear, the contribution
$$C(j\mid T) =  \mathbb{E}(f(x_{T \cup \{j\}}, X_{(T \cup \{j\})^C}) - f(x_T, X_{T^C})) = \mathbb{E}(a_j x_j - a_j X_j) = a_j( x_j -\mathbb{E}( X_j))$$
does not depend on $T$.

### Shapley values with log transformation
Adding the log transformation, this simplification does not work anymore since
$$C(j\mid \mathcal{I}) = -\log P^{\text{rd}(N_{\mathcal{I} \cup \{j\}})} \{g(N) \geq g(n)\}+\log P^{\text{rd}(N_{\mathcal{I}})} \{g(N) \geq g(n)\} = \log
\frac{ P^{\text{rd}(N_{\mathcal{I}})} \{g(N) \geq g(n)\}}{P^{\text{rd}(N_{\mathcal{I} \cup \{j\}})} \{g(N) \geq g(n)\}}.$$
and even if $f$ is linear, there is no easy way to get rid of this fraction.

In [1]:
import numpy as np, pandas as pd, networkx as nx
from dowhy import gcm

X = np.random.uniform(low=-5, high=5, size=1000)
Y = 0.5 * X + np.random.normal(loc=0, scale=1, size=1000)
Z = 2 * Y + np.random.normal(loc=0, scale=1, size=1000)
W = 3 * Z + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z, W=W))

causal_model = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z'), ('Z', 'W')]))  # X -> Y -> Z -> W
gcm.auto.assign_causal_mechanisms(causal_model, data)
gcm.fit(causal_model, data)

/Users/daniela/anaconda3/envs/causal_reading/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fitting causal mechanism of node W: 100%|██████████| 4/4 [00:00<00:00, 150.61it/s]


In [2]:
X = np.random.uniform(low=-5, high=5)  # Sample from its normal distribution.
Y = 0.5 * X + 5  # Here, we set the noise of Y to 5, which is unusually high.
Z = 2 * Y
W = 3 * Z
anomalous_data = pd.DataFrame(data=dict(X=[X], Y=[Y], Z=[Z], W=[W]))  # This data frame consist of only one sample here.

attribution_scores = gcm.attribute_anomalies(causal_model, 'W', anomaly_samples=anomalous_data)
attribution_scores

Evaluate set function: 16it [00:00, 52.69it/s]


{'X': array([0.28073264]),
 'Y': array([7.30187141]),
 'Z': array([0.11284866]),
 'W': array([0.31124813])}

### Comparison
Compare results to the scores I would get when using the Shapley values as defined in "Feature relevance quantification in explainable AI: A causal problem".

In [11]:
from sklearn.linear_model import LinearRegression
from dowhy.gcm.util.general import variance_of_deviations
from dowhy.gcm.util.general import means_difference

In [10]:
X = np.random.uniform(low=-5, high=5, size=1000)
Y = 0.5 * X + np.random.normal(loc=0, scale=1, size=1000)
Z = 2 * Y + np.random.normal(loc=0, scale=1, size=1000)
W = 3 * Z + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z, W=W))
# add anomalous row
X_outlier = np.random.uniform(low=-5, high=5)  # Sample from its normal distribution.
Y_outlier = 0.5 * X_outlier + 5  # Here, we set the noise of Y to 5, which is unusually high.
Z_outlier = 2 * Y_outlier
W_outlier = 3 * Z_outlier
W = np.append(W, W_outlier)
anomalous_data = pd.DataFrame(data=dict(X=[X_outlier], Y=[Y_outlier], Z=[Z_outlier], W=[W_outlier]))  # This data frame consist of only one sample here.
data = pd.concat([data, anomalous_data], ignore_index=True)

mdl = LinearRegression()
mdl.fit(data[['X', 'Y', 'Z']].to_numpy(), W)
relevance = gcm.feature_relevance_distribution(mdl.predict, data[['X', 'Y', 'Z']].to_numpy(), subset_scoring_func=variance_of_deviations)
relevance

Evaluate set function: 8it [00:00, 62.08it/s]


array([ -0.50471089,   2.00625383, 120.95873524])

In [13]:
single_observation = anomalous_data[['X', 'Y', 'Z']].to_numpy()
relevance = gcm.feature_relevance_sample(mdl.predict, data[['X', 'Y', 'Z']].to_numpy(), baseline_samples=single_observation, subset_scoring_func=means_difference)
relevance

Evaluate set function: 8it [00:00, 135.86it/s]


array([[-1.66288235e-02,  5.65014022e-01,  3.18805726e+01]])